In [1]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, GRU

# utils
import os
import subprocess

In [2]:
parent_relative_path = ".."

In [3]:
colab = 'google.colab' in str(get_ipython())

In [4]:
if colab:
    print("Colab babe")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

    os.system('cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" /root/.kaggle/kaggle.json')
    subprocess.run(["git", "clone", "https://github.com/codefupanda/customer_interaction_summary.git"])
    os.system('cd customer_interaction_summary && make requirements && make data > logs.logs')
    parent_relative_path = "./customer_interaction_summary"

In [7]:
isear = pd.read_csv(parent_relative_path + '/data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [8]:
number_of_classes = len(isear.EMOT.unique())

In [9]:
maxlen = 1000
max_words = 10000

In [10]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [11]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [12]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [13]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

In [ ]:
## Model creation time

In [ ]:
## Model creation time

In [ ]:
model = Sequential()
model.add(Embedding(max_words, output_dim=50, input_length=maxlen))
model.add(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(Flatten())
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [15]:
model.fit(x_train, to_categorical(y_train),
          epochs=5,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Epoch 1/5
180/180 [==============================] - 201s 1s/step - loss: 1.8819 - accuracy: 0.2237 - val_loss: 1.5744 - val_accuracy: 0.4085
Epoch 2/5
180/180 [==============================] - 193s 1s/step - loss: 1.2569 - accuracy: 0.5427 - val_loss: 1.3160 - val_accuracy: 0.5190
Epoch 3/5
180/180 [==============================] - 182s 1s/step - loss: 0.8345 - accuracy: 0.7153 - val_loss: 1.3758 - val_accuracy: 0.5149
Epoch 4/5
180/180 [==============================] - 173s 960ms/step - loss: 0.5718 - accuracy: 0.8074 - val_loss: 1.4198 - val_accuracy: 0.5389
Epoch 5/5
180/180 [==============================] - 175s 971ms/step - loss: 0.4022 - accuracy: 0.8652 - val_loss: 1.5873 - val_accuracy: 0.5394


In [16]:
y_pred = model.predict_classes(x_test)
y_pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([3, 7, 4, ..., 4, 1, 3])

In [ ]:
confusion_matrix(y_test, y_pred)